In [ ]:
#| default_exp api

# API

To install notebook development environment run the following command:
`conda create env -p ./.venv --file conda.env.yml`

In [ ]:
#|hide
# Development libraries, not required for package.
from nbdev.showdoc import *

Included libraries

In [ ]:
#|export
# Imports required for package
from pingme.core import Card, PingMe, set_env_variables

In [ ]:
#|hide
# For development sets real config values which are not stored in git
import os
from pathlib import Path

if os.environ.get("PINGME_CONFIG_PATH") is None and Path("./config/config.env").is_file():
    os.environ["PINGME_CONFIG_PATH"] = "./config/config.env"

#|hide

check env vars. This service requires a properly set `config.env` to work.

In [ ]:
#|export
import os
set_env_variables(os.environ.get("PINGME_CONFIG_PATH", None))

In [ ]:
#|hide
for k, v in os.environ.items():
    # If ENV var starts with PINGME_ then print
    if k.startswith("PINGME_"):
        print(f"{k}={v}")

## Notebook Variables

Here you can see included potential variables and their ENV variable names.

#|hide

Values here are used for this service and are thus `#|export`.

In [ ]:
#|export
import json

PINGME_ENV_CARD_NAME = os.environ.get('PINGME_ENV_CARD_NAME',None)
PINGME_ENV_CARD_CONTEXT = json.loads(os.environ.get('PINGME_ENV_CARD_CONTEXT', '{}'))
PINGME_ENV_CARD_DIR = os.environ.get('PINGME_ENV_CARD_DIR', None)
PINGME_ENV_CARD_EXT = os.environ.get('PINGME_ENV_CARD_EXT', None)

# For reducing spam when doing dev work
SEND_EMAIL = True
SEND_WEBHOOK = True
SEND_LOGFILE = True

Used the [FastAPI Notebook](https://github.com/David-Lor/FastAPI_LightningTalk-Notebook/blob/master/FastAPI.ipynb) as a starting point to see how to run commands within Jupyter, then looked into running the service [inside the notebook](https://stackoverflow.com/questions/63833593/how-to-run-fastapi-uvicorn-in-google-colab) which worked but you couldn't get passed the cell. So I ended up using the [test client](https://fastapi.tiangolo.com/tutorial/testing/) which makes sense

In [ ]:
#|export
import uvicorn
from fastapi import FastAPI
from fastapi.testclient import TestClient

app = FastAPI()

@app.get("/")
async def read_main():
    # Using the default place as a information post
    return {"msg": "PingMe: please POST to /webhook or /email or /logfile}"}

In [ ]:
client = TestClient(app)
response = client.get("/")
print(response.status_code, response.json())

In [ ]:
#|export
from fastapi import HTTPException
@app.post("/webhook")
async def send_webhook(card: Card):
    try:
        notification = PingMe(card, PINGME_ENV_CARD_DIR, PINGME_ENV_CARD_EXT)
        response = notification.send_webhook()
    except Exception as e:
        raise HTTPException(status_code=404, detail="Error in getting or sending card")
    return response

In [ ]:
client = TestClient(app)
response = client.post("/webhook", json={"name":"default", "context":{"title":"Title here", "text":"Text here"}})
print(response.json())

In [ ]:
#|export
from fastapi import HTTPException
@app.post("/email")
async def send_email(card: Card):
    try:
        notification = PingMe(card, PINGME_ENV_CARD_DIR, PINGME_ENV_CARD_EXT)
        response = notification.send_email()
    except Exception as e:
        raise HTTPException(status_code=404, detail=f"Error in getting or sending card {e}")
    return response

In [ ]:
client = TestClient(app)
response = client.post("/email", json={"name":"default", "context":{"title":"Title here", "text":"Text here"}})
print(response.json())

In [ ]:
#|export
from fastapi import HTTPException
@app.post("/logfile")
async def send_logfile(card: Card):
    try:
        notification = PingMe(card, PINGME_ENV_CARD_EXT, PINGME_ENV_CARD_EXT)
        response = notification.send_logfile()
    except Exception as e:
        raise HTTPException(status_code=404, detail="Error in getting or sending card")
    return response

In [ ]:
client = TestClient(app)
response = client.post("/logfile", json={"name":"default", "context":{"title":"Title here", "text":"Text here"}})
print(response.json())

For running the web service

In [ ]:
#|export
def web_service():
    uvicorn.run(app)

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()